In [14]:
import functools
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp

import Test_Likelihood as tl
import ODE_Dynamics as od
import Positive_Symptom_fn as fn


In [ ]:
tfd = tfp.distributions
tfb = tfp.bijectors

observations = test_data

In [15]:
def joint_log_prob(observations, R0, nu, i, s):
    
    Epi_Model = od.SIR
    epipar = tf.constant(np.array([[R0, 5E-8, nu, i, s]], dtype = np.float32))
    
    vdyn_ode_fn = od.ViralDynamics
    vpar = tf.constant(np.array([[5E3,0.01,5E-7,0.5,20,10]], dtype = np.float32))
    
    positive_fn = fn.proba_pos_sym(1.7).positive_fn, symptom_fn = fn.proba_pos_sym(1.7).symptom_fn
    prob_s_ibar = 0.1; prob_fp = 0.0
    duration = 100; Epi_cadence = 0.5; Vir_cadence = 0.0625
    
    par = tf.constant(np.array([[L,0.01,beta*1E-7,0.5,20,10, V0, X0, Y0]], dtype=np.float32))
    pospar = par; sympar = par
    
    loglike = tl.loglik(observations, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar, prob_fp, Epi_Model,
                 duration, Epi_cadence, Vir_cadence)
    
    return loglike.__call__(observations,epipar, vpar, pospar, sympar)

In [16]:
# compute log_lik imputing the things we don't want to sample, i.e observations
unnormalized_posterior_log_prob = functools.partial(joint_log_prob, observations)

NameError: name 'observations' is not defined

In [17]:
# Initialize parameters
r0 = 1.8; nu0 = 0.1; i0 = 0.001; s0 = 0.999
initial_state = [r0, nu0, i0, s0]

In [19]:
# Transform all parameters so that parameter spaces are unconstrained
# Every parameter needs a bijector, might be 'Identity'
unconstraining_bijectors = [
    tfb.Log(),
    tfb.Invert(tfb.Sigmoid()),
    tfb.Invert(tfb.Sigmoid()),
    tfb.Invert(tfb.Sigmoid())
    ]

In [20]:
@tf.function(autograph=False)
def sample():
    return tfp.mcmc.sample_chain(
        num_results = 2000,
        num_burnin_steps = 500,
        current_state = initial_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
                    target_log_prob_fn = unnormalized_posterior_log_prob,
                     step_size = 0.065,
                     num_leapfrog_steps = 5),
                bijector = unconstraining_bijectors),
             num_adaptation_steps = 400),
        trace_fn=lambda _, pkr: pkr.inner_results.inner_results.is_accepted)

[R0, nu, i, s], is_accepted = sample()

NameError: name 'unnormalized_posterior_log_prob' is not defined

In [12]:
# Compute posterior means of Epidemic parameters

acceptance_rate = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32)).numpy()
mean_R0 = tf.reduce_mean(R0, axis=0).numpy()
mean_nu = tf.reduce_mean(nu, axis=0).numpy()
mean_i = tf.reduce_mean(i, axis=0).numpy()
mean_s = tf.reduce_mean(s, axis=0).numpy()

NameError: name 'is_accepted' is not defined

In [13]:
print('acceptance_rate:', acceptance_rate)
print('avg basic reproduction number:', mean_R0)
print('avg recovery rate:', mean_nu)
print('avg infected', mean_i)
print('avg susceptible', mean_s)

NameError: name 'acceptance_rate' is not defined